In [103]:
import pandas as pd
import numpy as np
import networkx as nx

In [161]:
df = pd.read_csv("tcc_ceds_music.csv")
df = df.drop('Unnamed: 0',axis=1)

In [162]:
df

,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.000000
1,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,0.001284,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.000000
2,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.000000
3,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.000000
4,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,0.001350,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,78,0.001350,0.001350,0.001350,0.001350,...,0.065664,0.001350,0.889527,0.759711,0.062549,0.000000,0.751649,0.695686,obscene,0.014286
28368,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,67,0.001284,0.001284,0.035338,0.001284,...,0.001284,0.001284,0.662082,0.789580,0.004607,0.000002,0.922712,0.797791,obscene,0.014286
28369,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,77,0.001504,0.154302,0.168988,0.001504,...,0.001504,0.001504,0.663165,0.726970,0.104417,0.000001,0.838211,0.767761,obscene,0.014286
28370,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,67,0.001196,0.001196,0.001196,0.001196,...,0.001196,0.001196,0.883028,0.786888,0.007027,0.000503,0.508450,0.885882,obscene,0.014286


In [163]:
features = ['len', 'dating', 'violence', 'world/life', 'night/time',
       'shake the audience', 'family/gospel', 'romantic', 'communication',
       'obscene', 'music', 'movement/places', 'light/visual perceptions',
       'family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability',
       'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy', 'age']

In [164]:
for feature in features:
    df[feature] = pd.qcut(df[feature], q=3, labels=["low","medium","high"])

In [166]:
#list(set(df["artist_name"].unique()))

In [206]:
def construct_graph(df_, features_):
    features = features_.copy()
    df = df_.copy()
    
    graph = nx.Graph()
    
    for genre in list(set(df["genre"].unique())):
        graph.add_node(f"g:{genre}", ntype='category')
    
    for artist in list(set(df["artist_name"].unique())):
        graph.add_node(f"a:{artist}", ntype='category')
    
    for feature in features:
        graph.add_node(f"{feature}-low", ntype='category')
        graph.add_node(f"{feature}-medium", ntype='category')
        graph.add_node(f"{feature}-high", ntype='category')
        
    for idx, row in df.iterrows():
        graph.add_node(f"{idx}-song", ntype='song', name=row['track_name'], artist=row['artist_name'])
        graph.add_edge(f"{idx}-song", f"a:{row['artist_name']}",weight = 0.01)
        graph.add_edge(f"{idx}-song", f"g:{row['genre']}", weight = 3)
        
        for feature in features:
            graph.add_edge(f"{feature}-{row[feature]}",f"{idx}-song",weight = 1)
        
    return graph

In [207]:
G = construct_graph(df, features)

In [208]:
len(G.nodes())

33877

In [209]:
personalization_dict = {
    '27467-song': 1, #Imagine dragons - birds
    '23520-song': 1, #The kingston trio - the long black rifle
    '24077-song': 1, #Eagles - I wish you peace
}

In [210]:
rankings = nx.pagerank(G, personalization=personalization_dict, weight='weight')

In [211]:
#nx.get_node_attributes(G, 'ntype')

In [212]:
rankings = [t for t in sorted(rankings.items(), key=lambda x: x[1], reverse=True)]

In [213]:
playlist = []
plength = 30
i = 0
for t in rankings:
    if nx.get_node_attributes(G,'ntype')[t[0]] == 'song':
        playlist.append(f"{nx.get_node_attributes(G,'name')[t[0]]} - {nx.get_node_attributes(G,'artist')[t[0]]}")
        i+=1
        if i >=30:
            break

In [214]:
playlist

['i wish you peace - eagles',
 'the long black rifle - the kingston trio',
 'birds (feat. elisa) - imagine dragons',
 'forgive me - pat boone',
 'end of the night - the doors',
 'i go to sleep - the kinks',
 'julia - the beatles',
 'tequila sunrise - eagles',
 'tears - rush',
 'planets of the universe - fleetwood mac',
 'doing all right - queen',
 "i'm falling in love tonight - elvis presley",
 'oh, sail away - the kingston trio',
 'when my love was here - the kingston trio',
 'wimoweh - the kingston trio',
 'not today - imagine dragons',
 'merry christmas darling - carpenters',
 'no more - elvis presley',
 'reminder - mumford & sons',
 'love that burns - fleetwood mac',
 'hawaiian wedding song - elvis presley',
 'phenomenal cat - the kinks',
 'love will keep us alive - eagles',
 'i was the one - elvis presley',
 'birds - neil young',
 'starting today - elvis presley',
 'first in line - elvis presley',
 'if i keep my heart out of sight - james taylor',
 'surrender - elvis presley',
 'e